In [42]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
addr_translate = {
    'DR': 'DRIVE',
    'PKWY': 'PARKWAY',
    'RD': 'ROAD',
    'ST': 'STREET',
    'STE': 'SUITE',
    'N': 'NORTH',
    'S': 'SOUTH',
    'E': 'EAST',
    'W': 'WEST',
    'NE':'NORTHEAST',
    'LN': 'LANE',
    'CTR': 'CENTER',
    'CT': 'COURT',
    'BLVD': 'BOULEVARD',
    'CIR': 'CIRCLE',
    'HTS': 'HEIGHTS',
    'AVE': 'AVENUE',
    'HWY': 'HIGHWAY',
    'JCT': 'JUNCTION',
    'LK': 'LAKE',
    'MTN': 'MTN',
    'APT': 'APARTMENT',
    'RM': 'ROOM',
    'PL': 'PLACE',
    'PLZ': 'PLACA',
    'RDG': 'RIDGE',
    'SQ': 'SQUARE',
    'STA': 'STATION',
    'TER': 'TERRACE',
    'TRL': 'TRAIL',
    'TPKE': 'TURNPIKE',
    'VLY': 'VALLEY',
    'IS': 'ISLAND',
    '1ST': 'FIRST',
    '2ND':'SECOND',
    '3RD': 'THIRD',
    '4TH': 'FOURTH',
    '5TH': 'FIFTH',
    '6TH': 'SIXTH',
    '7TH': 'SEVENTH',
    '8TH': 'EIGHTH',
    '9TH':'NINTH',
    'ONE': '1',
}


def translate_addr(addr_string):
    addr_string = addr_string.replace('.','')
    addr_string = addr_string.upper()
    new_addr = []
    
    tokens = addr_string.split()
    
    for t in tokens:
        if t in addr_translate:
            new_addr.append(addr_translate[t])
        else:
            new_addr.append(t)
    
    return ' '.join(new_addr)

def clean_addr(addr_string):
    addr_string = addr_string.replace('#','')
    addr_string = addr_string.split(',')[0]
    addr_string =  translate_addr(addr_string)
    return(addr_string)

In [123]:
npi =pd.read_csv('npi_test.csv')

In [8]:
adds =[]
for add in list(npi['LOCATION_ADDRESS_1']):
    adds.append(clean_addr(add))

In [124]:
npi['ADDRESS']=adds

In [39]:
npi.groupby('ADDRESS').count().sort_values('NPI', ascending=False)[:751]

,NPI,Taxonomies,NAME,OTHER_NAME,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,MAIING_ZIP,MAILING_PHONE,LOCATION_ADDRESS_1,LOCATION_ADDRESS_2,LOCATION_CITY,LOCATION_STATE,LOCATION_ZIP,LOCATION_PHONE,LAST_UPDATE_DATE
ADDRESS,,,,,,,,,,,,,,,,,
1 MEDICAL CENTER DRIVE,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
100 HOSPITAL DRIVE,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
34800 BOB WILSON DRIVE,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
4200 TWELVE OAKS DRIVE,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
101 BODIN CIRCLE,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322 WEST SOUTH STREET,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1370 WEST D STREET,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
55 FOGG ROAD,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [21]:
addies =[]
for add in list(npi['ADDRESS']):
    good = False
    for letter in add:
        if letter.isnumeric()==True:
            good=True
            break
    if good == False:
        addies.append(add)

In [22]:
len(addies)

219

In [122]:
npi

1033239991

In [125]:
address_dict ={}
for row in npi[npi.NPI.isin(neeps)].itertuples():
    un = True
    if row.ADDRESS not in address_dict.keys():
        for address in address_dict.keys():
            if row.LOCATION_STATE == address_dict[address]['STATE']:
                if fuzz.ratio(row.ADDRESS, address)>90:
                    print(f'{row.ADDRESS} matches {address}')
                    address_dict[address]['NPIs'].append(row.NPI)
                    address_dict[address]['NAMES'].append(row.OTHER_NAME)
                    address_dict[address]['NAMES'].append(row.NAME)
                    print(address_dict[address]['NAMES'])
                    print(F"{address_dict[address]['ZIPCODE']}, {row.LOCATION_ZIP}")
                    un = False
                    print('---')
                    break
    else:
        print(f'{row.ADDRESS} already in dictionary!')
        address_dict[row.ADDRESS]['NPIs'].append(row.NPI)
        address_dict[row.ADDRESS]['NAMES'].append(row.OTHER_NAME)
        address_dict[row.ADDRESS]['NAMES'].append(row.NAME)
        print('---')
    if un == True:
        address_dict[row.ADDRESS] = {}
        address_dict[row.ADDRESS]['NPIs']=[row.NPI]
        address_dict[row.ADDRESS]['STATE']=row.LOCATION_STATE
        address_dict[row.ADDRESS]['NAMES']=[row.NAME]
        address_dict[row.ADDRESS]['NAMES'].append(row.OTHER_NAME)
        address_dict[row.ADDRESS]['ZIPCODE']=row.LOCATION_ZIP
    

901 HEARTLAND ROAD already in dictionary!
---
155 KNOLLCROFT ROAD matches 151 KNOLLCROFT ROAD
['VETERAN HOSPITAL', 'None', 'V.A. N.J. HEALTH CARE SYSTEM', 'U.S. DEPT. OF VETERANS AFFAIRS']
079395001, 079395001
---
2200 BERGQUIST DRIVE matches 2200 BERQUIST DRIVE
['USAF', 'WILFORD HALL MEDICAL CENTER', 'BROOKS HEALTH AND WELLNESS CENTER', 'BROOKS CITY BASE CLINIC']
782365300, 782369907.0
---
7600 CARROLL AVENUE already in dictionary!
---
1501 SAN PEDRO DRIVE SE already in dictionary!
---
2 FORDHAM HILL OVAL matches 7 FORDHAM HILL OVAL
['HHC: NORTH CENTRAL BRONX HOSPITAL', 'None', 'None', 'LINCOLN MEDICAL AND MENTAL HEALTH CENTER']
104684853, 104684704.0
---
101 BROAD STREET already in dictionary!
---
305 FIFTH & BERING STREET matches 305 FIFTH BERING STREET
['NORTON SOUND HEALTH CORP.', 'NORTON SOUIND REGIONAL HOSP.', 'NORTON SOUND REGIONAL HOSPITAL', 'NORTON SOUND HEALTH CORP']
99762.0, 99762.0
---
310 WEST LOSEY STREET already in dictionary!
---
801 GOODYEAR BOULEVARD already in dicti

101 THE CITY DRIVE SOUTH already in dictionary!
---
3000 SCHATULGA ROAD already in dictionary!
---
3515 BROADWAY AVENUE already in dictionary!
---
80 HOSPITAL DRIVE matches 101 HOSPITAL DRIVE
['CALDWELL COUNTY HOSPITAL, INC', 'None', 'None', 'MARY BRECKINRIDGE HEALTHCARE, INC.', 'KNOX COUNTY HOSPITAL', 'PACER MANAGEMENT OF KENTUCKY LLC']
424452301.0, 409067363
---
720 HOSPITAL DRIVE matches 2701 HOSPITAL DRIVE
['CITIZENS MEDICAL CENTER COUNTY OF VICTORIA', 'COUNTY OF VICTORIA CITIZENS MEDICAL CENTER', 'PERMIAN REGIONAL MEDICAL CENTER', 'ANDREWS COUNTY HOSPITAL DISTRICT']
779015748, 797143617.0
---
800 WASHINGTON STREET already in dictionary!
---
1824 MURDOCH AVENUE already in dictionary!
---
120 EAST HARRIS AVENUE already in dictionary!
---
2000 OGDEN AVENUE already in dictionary!
---
1431 NORTH CLAREMONT AVENUE already in dictionary!
---
923 EAST CENTRAL AVENUE matches 905 EAST CENTRAL AVENUE
['LAFOLLETTE COMM HOSPITAL', 'None', 'None', "ST. MARY'S MEDICAL CENTER OF CAMPBELL COUNTY, I

112 MANSFIELD AVENUE already in dictionary!
---
135 AVENUE G already in dictionary!
---
7620 SOUTHERN BOULEVARD matches 7630 SOUTHERN BOULEVARD
['THE SURGERY CENTER AT SOUTHWOODS, LLC', 'THE SURGICAL HOSPITAL AT SOUTHWOODS', 'SOUTHWOODS PHYSICAL THERAPY', 'SOUTHWOODS REHABILITATION LLC']
445125633, 445125667
---
30 SOUTH MAIN STREET matches 832 SOUTH MAIN STREET
['ORRVILLE HOSPITAL FOUNDATION', 'AULTMAN ORRVILLE HOSPITAL', 'SOUTHWOODS PHYSICAL THERAPY', 'SOUTHWOODS REHABILITATION LLC']
446672208.0, 445141914
---
7502 GREENVILLE AVENUE matches 7115 GREENVILLE AVENUE
['TEXAS INSTITUTE FOR SURGERY, LLP', 'TEXAS INSTITUTE FOR SURGERY AT TEXAS HEALTH PRESBYTERIAN DALLAS', 'WALNUT HILL MEDICAL CENTER', 'WALNUT HILL PHYSICIANS HOSPITAL, LLC']
752315101, 75231
---
2776 PACIFIC AVENUE already in dictionary!
---
201 LYONS AVENUE already in dictionary!
---
403 EAST MADISON STREET already in dictionary!
---
901 EAST 104TH STREET matches 601 EAST 14TH STREET
['BOTHWELL REGIONAL HEALTH CENTER', 'Non

12300 MCCRACKEN ROAD already in dictionary!
---
2055 NORTH MAIN STREET matches 355 NORTH MAIN STREET
['KANE COUNTY HUMAN RESOURCE SPECIAL SERVICE DISTRICT', 'KANE COUNTY HOSPITAL', 'MOUNTAIN WEST MEDICAL CENTER', 'TOOELE HOSPITAL CORPORATION']
847413260, 840749819
---
1324 NORTH SHERIDAN ROAD already in dictionary!
---
1111 11TH STREET matches 1118 11TH STREET
['GENESIS HEALTH SYSTEM', 'GENESIS MEDICAL CENTER DEWITT', 'HAWARDEN COMMUNITY HOSPITAL', 'COMMUNITY HOSP', 'None', 'HAWARDEN REGIONAL HEALTHCARE']
527421235, 510231903
---
909 AVENUE TITO CASTRO matches 917 AVENUE TITO CASTRO
['PONCE SCHOOL OF MEDICINE', 'None', 'None', 'SOUTHERN HOSPITALIST SERVICES PSC']
007164717, 00716
---
421 EAST STATE HIGHWAY 114 already in dictionary!
---
2001 NORTH OREGON STREET matches 2311 NORTH OREGON STREET
['IHS ACQUISITION NO 171 INC', 'MESA HILLS SPECIALTY HOSPITAL', 'THE HOSPITALS OF PROVIDENCE MEMORIAL CAMPUS', 'TENET HOSPITALS LIMITED']
799023216, 799023320.0
---
3200 CANYON LAKE DRIVE matches

401 EAST VAUGHN AVENUE already in dictionary!
---
2829 EAST HIGHWAY 76 already in dictionary!
---
275 HOSPITAL DRIVE matches 7500 HOSPITAL DRIVE
['SACRAMENTO RADIOLOGY MEDICAL GROUP, INC.', 'None', 'ADVENTIST HEALTH UKIAH VALLEY', 'UKIAH ADVENTIST HOSPITAL']
95823.0, 954824531
---
501 SOUTH RAGSDALE STREET already in dictionary!
---
111 HUNTOON MEMORIAL HIGHWAY already in dictionary!
---
2000 SOUTH PALESTINE STREET already in dictionary!
---
117 NORTH WINNSBORO STREET already in dictionary!
---
2701 US HIGHWAY 271 NORTH already in dictionary!
---
26300 SOUTH HIGHWAY 125 already in dictionary!
---
36488 SOUTH HIGHWAY 82 already in dictionary!
---
3181 SW SAM JACKSON PARK ROAD already in dictionary!
---
909 SUMNER STREET already in dictionary!
---
1515 COMMONWEALTH AVENUE matches 1959 COMMONWEALTH AVENUE
['ST ELIZABETHS MEDICAL CENTER', 'None', 'None', 'CURAHEALTH BOSTON, LLC']
021355924, 021353617
---
1711 WEST TEMPLE STREET already in dictionary!
---
409 COTTAGE ROAD already in diction

1840 AMHERST STREET already in dictionary!
---
200 MADISON AVENUE matches 100 MADISON AVENUE
['ATLANTIC HEALTH', 'None', 'ATLANTIC REHABILITATION INSTITUTE', 'ATLANTIC REHABILITATION INSTITUTE, LLC']
079606136, 07940
---
1711 WEST TEMPLE STREET already in dictionary!
---
1 MEDICAL PARK DRIVE already in dictionary!
---
10628 PARK ROAD already in dictionary!
---
17651 B HIGHWAY already in dictionary!
---
1521 GULL ROAD already in dictionary!
---
2400 NORTH ROCKTON AVENUE already in dictionary!
---
1823 COLLEGE AVENUE matches 1829 COLLEGE AVENUE
['MANHATTAN SURGICAL HOSPITAL LLC', 'MANHATTAN SURGICAL HOSPITAL', 'None', 'ASCENSION VIA CHRISTI HOSPITAL MANHATTAN, INC.']
665023381, 66502
---
539 EAST PRUDHOMME STREET already in dictionary!
---
2151 WEST SPRING STREET already in dictionary!
---
805 PAMPLICO HIGHWAY already in dictionary!
---
45 STREET LAWRENCE DRIVE matches 50 STREET LAWRENCE DRIVE
['VOA REHABILITATION CENTERS, INC.', 'SOJOURN AT SENECA', 'MERCY HEALTH - TIFFIN HOSPITAL', 'ME

11711 LIVINGSTON ROAD already in dictionary!
---
350 FISHER ROAD matches 130 FISHER ROAD
['CENTRAL VERMONT MEDICAL CENTER INC', 'CENTRAL VERMONT MEDICAL CENTER INC.', 'VERMONT PSYCHIATRIC CARE HOSPITAL', 'STATE OF VERMONT-DEPARTMENT OF MENTAL HEALTH']
056029516, 056337901
---
19211 MCKAY BOULEVARD already in dictionary!
---
1401 STREET JOSEPH PARKWAY matches 1315 STREET JOSEPH PARKWAY
['PEDIATRIX MEDICAL GROUP', 'None', 'None', 'THE METHODIST HOSPITAL', 'ST. JOSEPH MEDICAL CENTER', 'SJ MEDICAL CENTER, LLC']
770028233, 770028301
---
1460 ORANGE STREET already in dictionary!
---
15000 GRATIOT AVENUE already in dictionary!
---
3066 NORTH KENTUCKY STREET already in dictionary!
---
4401 UNION STREET matches 4401A UNION STREET
['ADVANCED CARE HOSPITAL OF NORTHERN COLORADO LLC', 'NORTHERN COLORADO LONG TERM ACUTE HOSPITAL', 'None', 'NORTHERN COLORADO REHABILITATION HOSPITAL INC']
805372800, 805372800
---
401 12TH STREET NORTH already in dictionary!
---
763 JOHNSONBURG ROAD already in dictiona

In [45]:
fuzz.ratio('4135 48TH STREET','2840 48TH STREET')

81

In [53]:
fuzz.ratio('60651', '60655')

80

In [95]:
len(address_dict.keys())

7401

In [61]:
address_dict

{'309 SOUTH 49TH AVENUE': {'NPIs': [1891815684],
  'STATE': 'NE',
  'NAMES': ['UNMC', 'None'],
  'ZIPCODE': '681323503'},
 '338 PRESIDENT STREET': {'NPIs': [1083733018],
  'STATE': 'NJ',
  'NAMES': ['AGS SERVICES', 'ALL GODS CHILDREN SERVICES'],
  'ZIPCODE': '076636323'},
 '800 IRVING AVENUE': {'NPIs': [1447272455],
  'STATE': 'NY',
  'NAMES': ['VA MEDICAL CENTER', 'None'],
  'ZIPCODE': '132102716.0'},
 '315 EAST BROADWAY': {'NPIs': [1063401859],
  'STATE': 'KY',
  'NAMES': ['GATEWAY AT NORTON REHABILITATION HOSPITAL, LLC',
   'GATEWAY REHABILITATION HOSPITAL AT NORTON HEALTHCARE PAVILION'],
  'ZIPCODE': '402021703'},
 '15000 GRATIOT AVENUE': {'NPIs': [1699926113],
  'STATE': 'MI',
  'NAMES': ['BCA OF DETROIT, LLC', 'STONECREST CENTER'],
  'ZIPCODE': '482051973.0'},
 '227 MCKINLEY AVENUE': {'NPIs': [1013954460],
  'STATE': 'MN',
  'NAMES': ['EVELETH MENTAL HEALTH SERVICES SOCS', 'None'],
  'ZIPCODE': '557341606'},
 '1305 WEST CHEROKEE STREET': {'NPIs': [1144268723],
  'STATE': 'OK',
  

In [76]:
words = [
'HOSPITAL',
'MEDICAL GROUP',
'REHAB',
'BIRTH CENTER',
'PSYCHIATRIC CENTER',
'HEALTHCARE',
'WOUND CARE',
'ADDICTION CENTER',
'HEALTH CENTER',
'MEDICAL CENTER',
'URGENT CARE',
'DEVELOPMENT CENTER',
'FACILITY',
'HEALTH',
'CENTER',
'SYSTEM',
'CLINIC',
'INSTITUTE',
'MEDICINE'
]


In [92]:
count =0
neeps =[]
for row in npi.sort_values(by ='NAME').itertuples():
    keep= False
    for word in words:
        if word in row.NAME:
            keep = True
            break
        elif word in row.OTHER_NAME:
            keep= True
            break
    if keep== False:
        print(f'{row.NAME} / {row.OTHER_NAME}')
    if keep == True:
        neeps.append(row.NPI)

1 MEB / None
121ST CSH/BAACH / ACH BRIAN ALLGOOD-SEOUL
1422 CAMINITO SEPTIMO / None
1958 / None
1ST ASSIST INC / None
1ST ASSIST OF RIVERSIDE MEDICAL CORPORATION, INC. / None
2045 FRANKLIN ST. / None
3853 ROSECRANS ST / None
412 RITTENHOUSE ST NW WASHINGTON DC 20011 / None
452 ASTS / None
60 MSGS / None
633 / None
66SCHULTZ PA / None
81MSGS/SGCQ / None
999999 / 999999
A BEAUTIFUL YOU / None
A BUITRAGO SURGICAL / None
A NURSE OF YOUR OWN & CO RECRUITMENT FIRM LLC / None
A PLUS PHYSICIANS LLC / None
A PRIVATE PLACE LLC / None
A&A PHYSICIANS P.C. / None
A.WHITEBREAD RNFA / None
AAA PSYCHIATRIC SERVICES, PLLC / None
AAC MEDICAL STAFF / None
AARM, PLLC / None
AARON ADLER / None
AARUSH MANCHANDA / None
ABBY E MCMAHON FNP PC / None
ABC DENVER LIMOUSINE LLC / None
ABS LINCS KY, LLC / CUMBERLAND HALL
ABS LINCS TN, INC / CUMBERLAND HALL
ABSOLUTE CARE, LLC / None
ACT & ICM/SCM / None
ACTIVE NEUROMONITORING LLC / None
ACUTE CARE TRANSITIONS, LLP / None
ADAM LAWRENCE / None
ADMHN / None
ADMINISTRAC

NORTH CAROLINA HEART & VASCULAR / None
NORTH CHICAGO VA / None
NORTH DALLAS WOMEN'S CARE PA / None
NORTH HOUSTON EXTREMITY CARE, PLLC / None
NORTH SHORE LIJ / None
NORTH SHORE LIJ / None
NORTH SHORE LIJ / None
NORTH SHORE LONG ISLAND JEWISH / None
NORTH TEXAS - MCA LLC / MEDICAL CITY ALLIANCE
NORTH TEXAS LONG TERM CARE , P.A. / None
NORTH-SHORE LIJ / None
NORTHCOAST ANESTHESIA PROVIDERS / None
NORTHCOAST PROFESSIONAL COMPANY LLC / None
NORTHEAST JOINT VENTURE, LLC / None
NORTHERN CALIFORNIA ANESTHESIA PHYSICIANS / None
NORTHERN KENTUCKY BEHAVIORAL, LLC / None
NORTHSHORE LIJ FOREST HILLS / None
NORTHSTAR ANESTHESIA / None
NORTHSTAR ANESTHESIA / None
NORTHSTAR ANESTHESIA / None
NORTHWELL / None
NORTHWEST ANESTHESIA, P.A. / None
NORTHWEST ASSOCIATION OF SURGICAL ASSISTANTS, INC. / NASA
NORTHWEST MEDICAL FOUNDATION OF TILLAMOOK / None
NORTHWEST MEDICAL FOUNDATION OF TILLAMOOK / None
NORTHWEST NEWBORN AND PEDIATRIC SERVICES PC / None
NORTHWEST PERMANENTE / None
NOVA CLINCARE, INC. / None
NO

In [93]:
count

0

In [85]:
bop = npi[npi.ADDRESS.isin(address_dict.keys())].drop_duplicates(['ADDRESS', 'LOCATION_STATE'], keep='last')

In [80]:
address_dict.keys()

dict_keys(['309 SOUTH 49TH AVENUE', '338 PRESIDENT STREET', '800 IRVING AVENUE', '315 EAST BROADWAY', '15000 GRATIOT AVENUE', '227 MCKINLEY AVENUE', '1305 WEST CHEROKEE STREET', '433A BROADWAY', '1214 COOLIDGE STREET', '2545 SCHOENERSVILLE ROAD', '3023 BUNKER HILL STREET', '857 DEEP SPRINGS DRIVE', '5602 HEALTH CENTER DRIVE', '710 WATER STREET', '45 EAST MILLER ROAD', '3851 ROGER BROOKE DRIVE', '641 NORTH NEW BALLAS ROAD', '42ND AND EMILE', 'CONDOMINIO PLAZA GRANDE & AVENUE REYNOSA', '1676 EAST MCMURRAY BOULEVARD', '1250 EAST ALMOND AVENUE', 'STREET 2 KM 495 NO 46', '258 NORTH RON MCNAIR BOULEVARD', 'NORTH HIGHWAY 1', 'A CO 168TH MED BN(AS)', '3565 SOUTH MAGNOLIA AVENUE', '760 HOSPITAL CIRLE', 'CROUSE HOSPITAL', '57 KINGMAN STREET', '115 CASS AVENUE', '611 WEST LEE AVENUE', '5 AUDREA ROAD', '601 MAIN STREET', '508 WEST FIFTH STREET', '2949 WEST FRONT STREET', 'WEST MORGAN AVENUE', '2535 BANKSTONE DRIVE SW', 'MINNEAPOLIS VA MEDICAL CENTER', '2356 SUTTER STREET', 'PM&R (117)', '1945 WEST

In [91]:
npi[npi.NPI.isin(neeps)]

,NPI,Taxonomies,NAME,OTHER_NAME,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,MAIING_ZIP,MAILING_PHONE,LOCATION_ADDRESS_1,LOCATION_ADDRESS_2,LOCATION_CITY,LOCATION_STATE,LOCATION_ZIP,LOCATION_PHONE,LAST_UPDATE_DATE,ADDRESS
3,1063401859,['283X00000X'],"GATEWAY AT NORTON REHABILITATION HOSPITAL, LLC",GATEWAY REHABILITATION HOSPITAL AT NORTON HEAL...,315 E BROADWAY,None,LOUISVILLE,KY,402021703,5023158300,315 E BROADWAY,None,LOUISVILLE,KY,402021703,5023158300,2007-07-08,315 EAST BROADWAY
5,1013954460,['283Q00000X'],EVELETH MENTAL HEALTH SERVICES SOCS,None,PO BOX 64979,None,SAINT PAUL,MN,551640979,6514313676,227 MCKINLEY AVE,None,EVELETH,MN,557341606,2184855000,2007-07-08,227 MCKINLEY AVENUE
6,1144268723,['282NR1301X'],LINDSAY MUNICIPAL HOSPITAL,None,PO BOX 888,None,LINDSAY,OK,730520888,4057561404,1305 W CHEROKEE ST,None,LINDSAY,OK,730525042,4057561404,2007-07-08,1305 WEST CHEROKEE STREET
7,1417075193,['282N00000X'],MASSACHUSETTS GENERAL HOSPITAL,None,15 PARKMAN ST,WACC 1-134,BOSTON,MA,021143117,None,433A BROADWAY,APT 76,SOMERVILLE,MA,021452314,8579284349,2007-07-08,433A BROADWAY
8,1235177197,['282N00000X'],"LHCG-III, LLC",LOUISIANA EXTENDED CARE HOSPITAL OF LAFAYETTE,420 W PINHOOK RD,SUITE A,LAFAYETTE,LA,705032131,3372331307,1214 COOLIDGE ST,8TH FLOOR,LAFAYETTE,LA,70505,3379844224,2007-07-08,1214 COOLIDGE STREET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10662,1184264798,['282N00000X'],PORTERCARE ADVENTIST HEALTH SYSTEM,None,PO BOX 910317,None,DENVER,CO,802910317,3037157000.0,7233 CHURCH RANCH BLVD STE 1028,None,WESTMINSTER,CO,800214094,3037157000.0,2020-01-11,7233 CHURCH RANCH BOULEVARD SUITE 1028
10665,1184802480,"['273Y00000X', '282N00000X']",KINGMAN HOSPITAL INC,KINGMAN REGIONAL MEDICAL CENTER - HUALAPAI MOU...,3269 STOCKTON HILL RD,None,KINGMAN,AZ,864093619,9287570626,3801 SANTA ROSA DR,None,KINGMAN,AZ,864012311.0,9282635688,2020-01-13,3801 SANTA ROSA DRIVE
10666,1932109048,['282NC0060X'],GUNNISON VALLEY HOSPITAL,None,711 N TAYLOR ST,None,GUNNISON,CO,812302296,9706411456,711 N TAYLOR ST,None,GUNNISON,CO,812302243,9706411456,2020-01-13,711 NORTH TAYLOR STREET
10667,1356331425,"['208M00000X', '282N00000X', '363LA2200X']",STAMFORD HOSPITAL,None,1 HOSPITAL PLZ,None,STAMFORD,CT,069023602,2032767463,1 HOSPITAL PLZ,None,STAMFORD,CT,069023602,2032767463,2020-01-13,1 HOSPITAL PLACA


In [138]:
DICT_LIST = []
for key in address_dict.keys():
    if len(address_dict[key]['NPIs']) > 1:
        name, index = choose_name(address_dict[key]['NAMES'])
        print(index)
        print(f"ALL NAMES: {address_dict[key]['NAMES']}")
        npi = address_dict[key]['NPIs'][index]
    else:
        name, index = choose_name(address_dict[key]['NAMES'])
        print(f"ALL NAMES: {address_dict[key]['NAMES']}")
        npi = address_dict[key]['NPIs'][0]
    print(name)
    print("")
    new_dict={
        'NPI': npi,
        'NAME':name,
        'ADDRESS': key,
        'STATE': address_dict[key]['STATE'],
        'ZIPCODE': address_dict[key]['ZIPCODE'][0:5]
    }
    DICT_LIST.append(new_dict)

ALL NAMES: ['GATEWAY AT NORTON REHABILITATION HOSPITAL, LLC', 'GATEWAY REHABILITATION HOSPITAL AT NORTON HEALTHCARE PAVILION']
Gateway Rehabilitation Hospital At Norton Healthcare Pavilion

ALL NAMES: ['BCA OF DETROIT, LLC', 'STONECREST CENTER']
Bca Of Detroit

ALL NAMES: ['EVELETH MENTAL HEALTH SERVICES SOCS', 'None']
Eveleth Mental Health Services Socs

ALL NAMES: ['LINDSAY MUNICIPAL HOSPITAL', 'None']
Lindsay Municipal Hospital

ALL NAMES: ['MASSACHUSETTS GENERAL HOSPITAL', 'None']
Massachusetts General Hospital

ALL NAMES: ['LHCG-III, LLC', 'LOUISIANA EXTENDED CARE HOSPITAL OF LAFAYETTE']
Louisiana Extended Care Hospital Of Lafayette

ALL NAMES: ['LEHIGH VALLEY HOSPITAL MUHLENBERG', 'None']
Lehigh Valley Hospital Muhlenberg

ALL NAMES: ['WEST TEXAS HOSPITAL, LTD', 'WEST TEXAS HOSPITAL']
West Texas Hospital

ALL NAMES: ['SID PETERSON MEMORIAL HOSPITAL', 'None']
Sid Peterson Memorial Hospital

ALL NAMES: ['NORTHERN INDIANA HEALTH CARE SYSTEM', 'None']
Northern Indiana Health Care Sys

Birth And Family Center For Community Wellness

ALL NAMES: ['NEW LIFE CENTERS', 'None']
New Life Centers

ALL NAMES: ['MS STATE HOSPITAL CRISIS INTERVENTION CENTER - CLEVENLAN', 'None']
Ms State Hospital Crisis Intervention Center - Clevenlan

ALL NAMES: ['HURLEY MEDICAL CENTER', 'None']
Hurley Medical Center

ALL NAMES: ['MS STATE HOSPITAL CRISIS INTERVENTION CENTER - GRENADA', 'None']
Ms State Hospital Crisis Intervention Center - Grenada

ALL NAMES: ['ORANGE REGIONAL MEDICAL CENTER', 'ARDEN HILL HOSPITAL']
Arden Hill Hospital

ALL NAMES: ['WELLNESS CARE MEDICAL CENTER INC.', 'None']
Wellness Care Medical Center Inc.

ALL NAMES: ['RIVER OAKS MEDICAL CENTER', 'TWELVE OAKS MEDICAL CENTER']
Twelve Oaks Medical Center

ALL NAMES: ['MONROE MEDICAL FOUNDATION, INC.', 'MONROE COUNTY MEDICAL CENTER']
Monroe Medical Foundation

ALL NAMES: ['STRONG MEMORIAL HOSPITAL', 'None']
Strong Memorial Hospital

ALL NAMES: ['OHIO VALLEY GENERAL HOSPITAL', 'None']
Ohio Valley General Hospital

ALL NAMES: 

Cedar Grove Animal Hospital

ALL NAMES: ['LOYOLA UNIVERSITY MEDICAL CENTER', 'None']
Loyola University Medical Center

ALL NAMES: ['VENTURA COUNTY MEDICAL CENTER PSYCHIATRIC UNIT', 'None']
Ventura County Medical Center Psychiatric Unit

ALL NAMES: ['DISTRICT HOSPITAL PARTNERS L P', 'THE GEORGE WASHINGTON UNIVERSITY HOSPITAL']
The George Washington University Hospital

ALL NAMES: ['SPARKS FAMILY HOSPITAL INC', 'NORTHERN NEVADA MEDICAL CENTER']
Sparks Family Hospital Inc

ALL NAMES: ['LAKEWOOD RANCH MEDICAL CENTER', 'LAKEWOOD RANCH MEDICAL CENTER']
Lakewood Ranch Medical Center

ALL NAMES: ['SUMMERLIN HOSPITAL MEDICAL CENTER L L C', 'SUMMERLIN HOSPITAL MEDICAL CENTER']
Summerlin Hospital Medical Center L L C

ALL NAMES: ['PINNACLE HEALTHCARE LLC', 'PINNACLE HOSPITAL']
Pinnacle Hospital

1
ALL NAMES: ['PONCE SCHOOL OF MEDICINE', 'None', 'None', 'SOUTHERN HOSPITALIST SERVICES PSC']
Southern Hospitalist Services Psc

ALL NAMES: ['SISTERS OF MERCY OF WILLARD, OHIO', 'MERCY HOSPITAL OF WILLAR

ALL NAMES: ['LOUISIANA COMMUNITY HEALTH AND WELLNESS CENTER LLC', 'None']
Louisiana Community Health And Wellness Center Llc

ALL NAMES: ['USC DEPARTMENT OF SURGERY', 'USC UNIVERSITY HOSPITAL']
Usc University Hospital

ALL NAMES: ['SAINT FRANCIS HOSPITAL', 'None']
Saint Francis Hospital

ALL NAMES: ['GA DEPARTMENT OF HUMAN RESOURCES', 'GEORGIA REGIONAL HOSPITAL AT ATLANTA']
Georgia Regional Hospital At Atlanta

ALL NAMES: ['AMERICAN HOSPITAL SERVICES GROUP', 'None']
American Hospital Services Group

ALL NAMES: ['TENDERCARE MICHIGAN INC', 'ROGERS CITY REHABILITATION HOSPITAL']
Rogers City Rehabilitation Hospital

ALL NAMES: ['SOUTHSIDE PET HOSPITAL, INC.', 'None']
Southside Pet Hospital

ALL NAMES: ["DANVILLE CHILDREN'S MEDICAL CENTER", 'None']
Danville Children'S Medical Center

ALL NAMES: ['QUITMAN COUNTY HOSPITAL, LLC', 'None']
Quitman County Hospital

ALL NAMES: ['CLARK REGIONAL MEDICAL CENTER, INC', 'None']
Clark Regional Medical Center

ALL NAMES: ['ACADIA-ST. LANDRY HOSPITAL SERV


ALL NAMES: ['NEWBERRY COUNTY MEMORIAL HOSPITAL', 'None']
Newberry County Memorial Hospital

ALL NAMES: ['DUKE UNIVERSITY HOSPITAL', 'None']
Duke University Hospital

ALL NAMES: ['WESTERN PENNSYLVANIA HEALTH SYSTEM', 'None']
Western Pennsylvania Health System

ALL NAMES: ['HOSPITAL FOR SPECIAL SURGERY', 'None']
Hospital For Special Surgery

ALL NAMES: ['DOUGLAS COUNTY MEMORIAL HOSPITAL', 'None']
Douglas County Memorial Hospital

ALL NAMES: ['CHRIST MEDICAL CENTER', 'None']
Christ Medical Center

ALL NAMES: ['CHARLTON MEMORIAL HOSPITAL', 'None']
Charlton Memorial Hospital

ALL NAMES: ['PLANTATION GENERAL HOSPITAL L P', 'PLANTATION GENERAL HOSPITAL']
Plantation General Hospital L P

ALL NAMES: ['SACRED HEART HEALTH SERVICES', 'AVERA SACRED HEART HOSPITAL']
Avera Sacred Heart Hospital

1
ALL NAMES: ['TEXAS INSTITUTE FOR SURGERY, LLP', 'TEXAS INSTITUTE FOR SURGERY AT TEXAS HEALTH PRESBYTERIAN DALLAS', 'WALNUT HILL MEDICAL CENTER', 'WALNUT HILL PHYSICIANS HOSPITAL, LLC']
Walnut Hill Physici

...

ALL NAMES: ['COMMUNITY BEHAVIORAL HEALTH, LLC', 'None']
Community Behavioral Health

0
ALL NAMES: ['MORGAN COUNTY GEORGIA HOSPITAL AUTHORITY', 'MORGAN MEMORIAL HOSPITAL', 'None', 'CHATUGE REGIONAL HOSPITAL INC']
Morgan County Georgia Hospital Authority

ALL NAMES: ['LOMPOC VALLEY MEDICAL CENTER', 'LOMPOC DISTRICT HOSPITAL']
Lompoc District Hospital

ALL NAMES: ['CENTRAL GEORGIA REHABILITATION', 'None']
Central Georgia Rehabilitation

ALL NAMES: ['IHC HEALTH SERVICES INC.', 'BEAR RIVER HOSPITAL']
Bear River Hospital

ALL NAMES: ['DIXIE REGIONAL MEDICAL CENTER', 'None']
Dixie Regional Medical Center

ALL NAMES: ['IHC HEALTH SERVICES INC', 'AMERICAN FORK HOSPITAL']
American Fork Hospital

ALL NAMES: ['IHC HEALTH SERVICES INC.', 'THE ORTHOPEDIC SPECIALTY HOSPITAL']
The Orthopedic Specialty Hospital

ALL NAMES: ['SAUNDERS MEDICAL CENTER', 'None']
Saunders Medical Center

ALL NAMES: ['TUOMEY', 'TUOMEY HEALTHCARE SYSTEM']
Tuomey Healthcare System

ALL NAMES: ['IHC HEALTH SERVICES, INC.', 'ORE


ALL NAMES: ['NYS OFFICE OF MENTAL HEALTH', 'ROCKLAND CHILDRENS PSYCHIATRIC CENTER']
Rockland Childrens Psychiatric Center

ALL NAMES: ['NYS OFFICE OF MENTAL HEALTH', 'ROCHESTER PSYCHIATRIC CENTER']
Rochester Psychiatric Center

ALL NAMES: ['NYS OFFICE OF MENTAL HEALTH', 'SAGAMORE CHILDRENS PSYCHIATRIC CENTER']
Sagamore Childrens Psychiatric Center

ALL NAMES: ['NYS OFFICE OF MENTAL HEALTH', 'MANHATTAN PSYCHIATRIC CENTER']
Manhattan Psychiatric Center

ALL NAMES: ['NYS OFFICE OF MENTAL HEALTH', 'CAPITAL DISTRICT PSYCHIATRIC CENTER']
Capital District Psychiatric Center

ALL NAMES: ['NYS OFFICE OF MENTAL HEALTH', 'WESTERN NEW YORK CHILDRENS PSYCHIATRIC CENTER']
Western New York Childrens Psychiatric Center

ALL NAMES: ['GENERAL HOSPITAL CORP', 'None']
General Hospital Corp

ALL NAMES: ['COLDS & CUDDLES CHILDCARE CENTER LLC', 'None']
Colds & Cuddles Childcare Center Llc

ALL NAMES: ['KINGMAN HOSPITAL INC', 'None']
Kingman Hospital Inc

ALL NAMES: ['DECATUR COUNTY MEMORIAL HOSPITAL', 'None

Forrest General Cancer Center

ALL NAMES: ['HUNT MEMORIAL HOSPITAL DISTRICT', 'HUNT REGIONAL COMMUNITY HOSPITAL']
Hunt Regional Community Hospital

ALL NAMES: ['OSBORNE COUNTY MEMORIAL HOSPITAL', 'None']
Osborne County Memorial Hospital

ALL NAMES: ['WAMEGO HOSPITAL ASSOCIATION', 'None']
Wamego Hospital Association

ALL NAMES: ['ACCORD MEDICAL MANAGEMENT, LP', 'NIX HEALTH CARE SYSTEM']
Accord Medical Management

ALL NAMES: ['SARATOGA HOSPITAL', 'None']
Saratoga Hospital

ALL NAMES: ['HOSPITAL DISTRICT NO. 1 OF DICKINSON COUNTY, KANSAS', 'MEMORIAL HOSPITAL']
Hospital District No. 1 Of Dickinson County

ALL NAMES: ['KIMBLE COUNTY HOSPITAL DISTRICT', 'None']
Kimble County Hospital District

ALL NAMES: ['BEVERLY HILLS SUNSET SURGERY CENTER, INC', 'None']
Beverly Hills Sunset Surgery Center

ALL NAMES: ['COUNTY OF MONTEREY', 'NATIVIDAD MEDICAL CENTER']
Natividad Medical Center

ALL NAMES: ['ILLINOIS DEPARTMENT OF HUMAN SERVICES', 'JOHN J MADDEN MENTAL HEALTH']
Illinois Department Of Human S

Baylor Scott & White Orthopedic And Spine Hospital

ALL NAMES: ['HCN EP HORIZON CITY, LLC', 'THE HOSPITALS OF PROVIDENCE NORTHEAST CAMPUS']
The Hospitals Of Providence Northeast Campus

ALL NAMES: ['VIRGINIA HOSPITAL CENTER ARLINGTON', 'None']
Virginia Hospital Center Arlington

ALL NAMES: ['VIRGINIA COMMONWEALTH UNIVERSITY HEALTH SYSTEM AUTHORITY', 'VIRGINIA TREATMENT CENTER FOR CHILDREN']
Virginia Commonwealth University Health System Authority

ALL NAMES: ['HARSHA BEHAVIORAL CENTER, INC', 'None']
Harsha Behavioral Center

ALL NAMES: ['PRESTIGE REHAB', 'ST. CATHERINE MEMORIAL HOSPITAL']
St. Catherine Memorial Hospital

ALL NAMES: ['LISBON AREA HEALTH SERVICES', 'CHI LISBON HEALTH']
Lisbon Area Health Services

ALL NAMES: ['ALTON MEMORIAL HOSPITAL', 'None']
Alton Memorial Hospital

ALL NAMES: ['PUBLIC HOSPITAL DIST NO 1 SKAGIT', 'SKAGIT VALLEY HOSPITAL']
Public Hospital Dist No 1 Skagit

ALL NAMES: ['FORT WORTH SURGICARE PARTNERS LTD', 'BAYLOR SURGICAL HOSPITAL AT FORT WORTH OUTPATIEN

Blue Mountain Hospital District

ALL NAMES: ['POTOMAC HOSPITAL CORPORATION OF PRINCE WILLIAM', 'SENTARA NORTHERN VIRGINIA MEDICAL CENTER']
Potomac Hospital Corporation Of Prince William

ALL NAMES: ['NORTHWEST FLORIDA HEALTHCARE, INC', 'None']
Northwest Florida Healthcare

ALL NAMES: ['SENTARA HOSPITALS', 'SENTARA VIRGINIA BEACH GENERAL HOSPITAL']
Sentara Virginia Beach General Hospital

ALL NAMES: ['SENTARA HOSPITALS', 'SENTARA WILLIAMSBURG REGIONAL MEDICAL CENTER']
Sentara Hospitals

ALL NAMES: ['SENTARA RMH MEDICAL CENTER', 'ROCKINGHAM MEMORIAL HOSPITAL']
Rockingham Memorial Hospital

ALL NAMES: ['SENTARA PRINCESS ANNE HOSPITAL', 'SENTARA HOSPITALS']
Sentara Princess Anne Hospital

ALL NAMES: ['ASPEN VALLEY HOSPITAL DISTRICT', 'None']
Aspen Valley Hospital District

ALL NAMES: ['MARTHA JEFFERSON HOSPITAL', 'None']
Martha Jefferson Hospital

ALL NAMES: ['CONWAY HOSPITAL INC', 'CONWAY MEDICAL CENTER']
Conway Hospital Inc

ALL NAMES: ['SENTARA HOSPITALS', 'SENTARA OBICI HOSPITAL']
Sent

In [98]:
nms = ['BLACKFEET COMMUNITY HOSPITAL','None','BF HOSPITAL','INDIAN HEALTH SERVICE','U S HEALTH DEPT OF HEALTH & HUMAN SERVICES']

In [150]:
def choose_name(NAMES):
    index = 0
    len_name =0
    if 'HOSPITAL' in str(NAMES):
        for name in NAMES:
            if 'HOSPITAL' in name and len(name) > len_name:
                chosen_name = name
                len_name = len(name)
                name_index = index
            index += 1
    else:
        for name in NAMES:
            if len(name) > len_name:
                chosen_name = name
                len_name = len(name)
                name_index = index
            index += 1
    chosen_name = chosen_name.split(',')[0]
    name_index =  int((name_index + 0.5)/2)
    return(chosen_name, name_index)

In [106]:
choose_name(nms)

('BLACKFEET COMMUNITY HOSPITAL', 0)

In [109]:
:
    print('y')

y


In [208]:
len(address_dict)

7401

In [120]:
nms.remove('BLACKFEET COMMUNITY HOSPITAL')

In [147]:
doop = pd.DataFrame(DICT_LIST)

In [134]:
int(1.5)

1

In [ ]:
addies =[]
for add in list(npi['ADDRESS']):
    good = False
    for letter in add:
        if letter.isnumeric()==True:
            good=True
            break
    if good == False:
        addies.append(add)

In [145]:
addies =[]
for item in DICT_LIST:
    good = False
    for letter in item['ADDRESS']:
        if letter.isnumeric()==True:
            good=True
            break
    if good == True:
        addies.append(item['NPI'])
        
        

In [146]:
len(addies)

7261

In [149]:
doop[doop['NPI'].isin(addies)].to_csv('NPI_HOSP_2.csv', index=False)

In [154]:
name_list = name.split(' ')
for word in name_list:
    if len(word)>4 or word in :
        word = word.title()
    if word in short_word_list

SyntaxError: invalid syntax (<ipython-input-154-fbec48c5fbd2>, line 3)

In [205]:
mispelled = []
for name in list(doop[doop['NPI'].isin(addies)]['NAME']):
    name_list = name.split(' ')
    wrorn = list(spell.unknown(name_list))
    if len(wrorn) >0:
        mispelled += wrorn

In [ ]:
for name in 

In [184]:
def fix_name(nm):
    name_list = nm.split(' ')
    new_word_list = []
    if name_list[0].upper()=='THE':
        new_word_list.append('The')
        name_list = name_list[1:]
    for word in name_list:
        if "'S" in word:
            word = word.split("'")
            new_word = word[0].title()+"'s"
        elif word.upper() in short_word_list:
            new_word = word.lower()
        elif len(word)<4 and word.upper() not in other_word_list:
            new_word = word.upper()
        else:
            numeric = False
            for letter in word:
                if letter.isnumeric()==True:
                    new_word = word.lower()
                    numeric = True
                    break
            if numeric == False:
                new_word = word.title()
        new_word_list.append(new_word)
    return ' '.join(new_word_list)

In [180]:
fix_name('THE HOSPITAL AT THE NEW 18TH STREET')

'The Hospital at the New 18th Street'

In [185]:
short_word_list = ['AT','FOR','OF', 'AND','IN','AN','DEL','DES','ON','THE']
other_word_list = ['INC', 'INC.','ST.','ST','NEW','LEE','THE','LOS','SAN','TRI','RED','TOP','OUR','NO','NO.','LA','MT',
                  'ALL','ANN','ANA','ARC','BAY','BED','BEN','BIG','BOX','CAT','DAY','DEE','DR.','EAR','ELK','FOX','HOT',
                   'IN','JAY','JOY','JR.','JR','KAU','KAY','KIM','KIT','LAB','LAS','LE','LEA','MED','MID','NET','NIX','NOT',
                   'OAK','ONE','OLD','PAM','PAZ','PEE','PEA','PET','RAY','RED','REG','REX','REY','ROY','RUN','RYE', 'SAN',
                   'SEA','SUB','SUN','SKY','TEN','TUG','TWO','VAN','WAR','WAY','SID','CHI']

In [174]:
name_list = ['we','are','not','free']

In [175]:
name_list[1:]

['are', 'not', 'free']

In [187]:
from spellchecker import SpellChecker

In [189]:
spell = SpellChecker()

# find those words that may be misspelled
# misspelled = spell.unknown(['something', 'is', 'hapenning', 'here'])

# for word in misspelled:
#     # Get the one `most likely` answer
#     print(spell.correction(word))

#     # Get a list of `likely` options
#     print(spell.candidates(word))

In [200]:
list(spell.unknown(['bee']))

[]

In [206]:
mispelled

['socs',
 'eveleth',
 'muhlenberg',
 'lehigh',
 "john's",
 'mid-carolina',
 "men's",
 '18th',
 'medbn',
 'blackfeet',
 'larue',
 'charlotte-mecklenburg',
 'debakey',
 'wausau',
 'aspirus',
 'spearfish',
 'pediatrix',
 'debakey',
 'sonora',
 'pinecrest',
 'univeristy',
 'vison',
 'altohealth',
 'lsu',
 'southpark',
 "women's",
 'rehabilitaton',
 'stroger',
 'evansville',
 'natchez',
 'integra',
 'lehigh',
 'barberton',
 'carbondale',
 'brookdale',
 "john's",
 'monsour',
 'creedmoor',
 'system-marion',
 'greenbriar',
 'ooonemourschildrenhospital',
 'scca-oral',
 'rehabvisions',
 'hendersonville',
 'subacute',
 'marymount',
 'hemodialysis',
 'richland',
 'flemingsburg',
 'itasca',
 'abilene',
 'christianacarehealthservices',
 'rehabilitatio',
 'fairview',
 'acadiana',
 'acadiana',
 'woodhull',
 'woodhull',
 'millcreek',
 'millcreek',
 'millcreek',
 "children's",
 'waukegan',
 'llc',
 'anadarko',
 'ozarks',
 'mesilla',
 'st.peters',
 'udh',
 'barberton',
 '1-a',
 'richland',
 'pediatrix',
